### 16-1 난수 생성

In [2]:
import numpy as np

def sn_random_numbers(shape,antithetic = True, moment_matching = True,
                     fixed_seed = False) :
    '''
    shape 인수와 같은 shpae를 가지면서 표준 정규분포를 따르는 (의사)난수 배열을 반환
    
    인수
    ----
    shape : tuple (o,n,m)
        shape (o,n,m) 형태의 배열 생성
    antithetic : Boolean
        대조변수 생성
    moment_matching : Boolean
        1차 및 2차 모멘트 정합
    fixed_seed : Boolean
        seed를 고정하기 위한 플래그
    
    반환값
    ----
    ran : (o,n,m) 형태의 의사 난수 배열
    '''
    if fixed_seed :
        np.random.seed(1000)
    if antithetic :
        ran = np.random.standard_normal((shape[0],shape[1],shape[2]/2))
        ran = np.concatenate((ran,-ran),axis = 2)
    else:
        ran = np.random.standard_normal(shape)
    if moment_matching  :
        ran = ran-np.mean(ran)
        ran = ran/np.std(ran)
    if shape[0] == 1 :
        return ran[0]
    else :
        return ran

In [4]:
snrn = sn_random_numbers((2,2,2),antithetic =False,
                        moment_matching = False,
                        fixed_seed = True)

In [5]:
snrn

array([[[-0.8044583 ,  0.32093155],
        [-0.02548288,  0.64432383]],

       [[-0.30079667,  0.38947455],
        [-0.1074373 , -0.47998308]]])

In [6]:
snrn_mm = sn_random_numbers((2,3,2),antithetic=False,
                           moment_matching= True,
                           fixed_seed= True)

In [7]:
snrn_mm

array([[[-1.47414161,  0.67072537],
        [ 0.01049828,  1.28707482],
        [-0.51421897,  0.80136066]],

       [[-0.14569767, -0.85572818],
        [ 1.19313679, -0.82653845],
        [ 1.3308292 , -1.47730025]]])

In [8]:
snrn_mm.mean(), snrn_mm.std()

(3.700743415417188e-17, 1.0)

### 16-2 Genaral Simulation Class

In [9]:
#DX Library Simulation
#simulation_class.py
import numpy as np
import pandas as pd

class simulation_class(object):
    '''
    given base method for simulation class
    
    속성
    ----
    name: string
        객체이름
    mar_env : instance of market_environment
        시뮬레이션에 필요한 시장환경자료
    corr : Boolean
        다른 모형 객채와 상관관계 있으면 True
    
    method
    ----
    generate_time_grid :
        시뮬레이션 시간 그리드 반환
    get_instrument_values :
        현재 증권 가치(배열) 변환
    '''
    def __init__(self,name,mar_env,corr):
        try:
            self.name = name
            self.pricing_date = mar_env.pricing_date
            self.initial_value = mar_env.get_constant('inital_value')
            self.volatility = mar_env.get_constant('volatility')
            self.final_date = mar_env.get_constant('final_date')
            self.currency = mar_env.get_constant('currency')
            self.frequency = mar_env.get_constant('frequency')
            self.paths = mar_env.get_constant('paths')
            self.discount_curve = mar_env.get_curve('discount_curve')
            try:
                # if mar_env include time_grid then return this value
                # 포폴 가치 평가용
                self.time_grid = mar_env.get_list('time_grid')
            except :
                self.time_grid = None
            try:
                # 만약 특별한 날짜 있으면 여기 추가
                self.special_dates = mar_env.get_list('special_dates')
            except:
                self.special_dates = []
            self.instrument_values = None
            self.correlated = corr
            if corr is True :
                #상관관계가 있는 포폴에만 필요
                self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
                self.rn_set = mar_env.get_list('rn_set')[self.name]
                self.random_numbers = mar_env.get_list('random_numbers')
        except:
            print('Error parsing market environmnet.')
            
    def generate_time_grid(self):
        start = self.pricing_date
        end = self.final_date
        #pandas date_range함수 ( day : B, week : W, month : M)
        time_grid = pd.date_range(start = start, end = end, freq = self.frequency).to_pydatetime()
        time_grid = list(time_grid)
        # 시작,끝,그리고 특별 날짜를 time_grid에 추가
        if start not in time_grid:
            time_grid.insert(0,start)
            #시작일 리스트에 없음 추가
        if end not in time_grid :
            time_grid.insert(end)
            #end 리스트에 없음 추가
        if len(self.special_dates) > 0 :
            #모든 특별 날짜 추가
            time_grid.extend(self.special_dates)
            #복제추가
            time_grid = list(set(time_grid))
            #list 정렬
            time_grid.sort()
        self.time_grid = np.array(time_grid)
    
    def get_instrument_values(self,fixed_seed = True):
        if self.instrument_values is None :
            #증권가 없음 시뮬레이션 시작
            self.generate_paths(fixed_seed = fixed_seed, day_count = 365.)
        elif fixed_seed is False:
            #다시 시뮬
            self.generate_paths(fixed_seed = fixed_seed, day_count = 365.)
        return self.instrument_values